In [ ]:
# !find ./log/* -type f -not -name "stepNum*" -exec rm -f {} \;
# clean the json format log and it's corresponding csv file

In [ ]:
import pandas
import matplotlib.pyplot as plt
import json
import matplotlib
# 设置字体为 Times New Roman
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Read the data
data = pandas.read_csv('./norm/norm10.csv', header=None)
data

In [ ]:
import pandas
import matplotlib.pyplot as plt
import json

class Log():
    def __init__(self, json_log_path):
        self.json_log_path = json_log_path
        self.param = json.load(open(self.json_log_path, 'r'))
        self.param_names = list(self.param.keys())
    def __getitem__(self, key):
        assert key in self.param.keys(), "key not in param"
        return self.param[key]
    
    def get_data(self,nrows=None):
        if nrows is None:
            return pandas.read_csv(self.param["data"])
        else:
            return pandas.read_csv(self.param["data"], nrows=nrows)
    
    def __str__(self) -> str:
        return f"Log({self.json_log_path})"
    
    def serialize(self) -> str:
        """
        序列化的时候忽略属性"other"
        """
        temp_json = self.param.copy()
        temp_json.pop("other")
        temp_json.pop("data")
        return json.dumps(temp_json)
    

# log1 = Log("./log/20231224063327_11cd96b4-ca81-4e25-ad9e-5aec0c3f3fa3.json")
# print(log1.param_names, log1["normId"])
# log1["data"]
# print(log1.serialize())
# log1.get_data()
# del log1

In [ ]:
import pandas
import os

NROWS = 200000
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
            print(os.path.join(root, file))
print(logs)

# 对logs按照norm的大小排序
logs = sorted(logs, key=lambda log:log["normId"])

# 限制读取行数为 100000 行
data = logs[0].get_data(NROWS)
data

In [ ]:
import pandas
import os

NROWS = 200000
logs = list()
for root, dirs, files in os.walk("./log/"):
    for file in files:
        if file.endswith(".json"):
            logs.append(Log(os.path.join(root, file))) 
print(logs)

# sort logs by normId
logs = sorted(logs, key=lambda log:log["normId"])

print("logs_num:", len(logs))
filtered_logs = list()
for log in logs:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 0.5:
    # if log["stepNum"] == 1000000 and log["mu"] == 0.0001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_longterm_no_norm_error":
    if log["stepNum"] == 50000 and log["mu"] == 0.001 and log["p0"] == 0.5 and log["payoffMatrix"] == "payoffMatrix_shortterm":
    # if log["stepNum"] == 100000 and log["mu"] == 0.001 and log["p0"] == 1.0 and log["payoffMatrix"] == "payoffMatrix_shortterm":
        print(log["data"])
        print(log.serialize())
        filtered_logs.append(log)
print("filtered_logs_num:", len(filtered_logs))

In [ ]:
logs = filtered_logs
end = NROWS
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    plt.figure(figsize=(11, 5))
    for i in range(1, 17):
        plt.plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[i - 1])
    # 刻度间隔
    x_major_locator = plt.MultipleLocator(5000)
    y_major_locator = plt.MultipleLocator(0.1)
    ax = plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)
    ax.yaxis.set_major_locator(y_major_locator)
    # 设置刻度
    margin = 0.02
    plt.xlim(-log["stepNum"] * margin, log["stepNum"] * (1 + margin))
    plt.ylim(-1*margin*2, 1 + margin*2)

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), ncol=8, fancybox=True, shadow=True)
    plt.xlabel('Time', fontsize=15)
    plt.ylabel('Donor-recipient Strategies', fontsize=15)
    plt.title("Social Norm Id: " + str(log["normId"]), fontsize=15)
    # plt.title("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    plt.tight_layout()
    plt.savefig('./temp_fig/shortterm/' + 'd-r_norm' + str(log["normId"]) + '.png', dpi = 600)

In [ ]:
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    plt.figure(figsize=(20, 4))
    fig, axes = plt.subplots(1,2,figsize=(20,6))
    fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    for i in range(17, 21):
        color_index = i - 17
        axes[0].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[color_index])
        axes[0].set_title("donor")
        axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=4, fancybox=True, shadow=True)
        axes[1].plot(data.iloc[:end, 0], data.iloc[:end, i+4], label=data.columns.values[i+4], color=colors[color_index])
        axes[1].set_title("recipient")
        axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.20), ncol=4, fancybox=True, shadow=True)
    # plt.savefig('./norm10-c-ur.png', dpi=600)

In [ ]:
# 画第25列
for log in logs:
    # 读取数据
    data = log.get_data(NROWS)
    # 第一列为横轴，第二列为纵轴画曲线，label为列名
    # 设置cmap
    colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple', 'pink', 'orange', 'gray', 'brown', 'cyan', 'magenta',
              'olive', 'lime', 'teal', 'navy']
    # plt.figure(figsize=(20, 6))
    fig, axes = plt.subplots(1,2,figsize=(15, 4))
    # fig.subtitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    fig.suptitle("normId: " + str(log["normId"]) + " payoffMatrix: " + str(log["payoffMatrix"]) + "\n" + log.serialize())
    fig.subplots_adjust(top=0.8)
    color_index = 0
    for i in range(25, 27):
        index = i - 25
        axes[index].set_title(data.columns.values[i])
        axes[index].set_xlabel(data.columns.values[0])
        axes[index].set_ylabel(data.columns.values[i])
        axes[index].plot(data.iloc[:end, 0], data.iloc[:end, i], label=data.columns.values[i], color=colors[color_index])
        color_index += 1
    # plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.30), ncol=4, fancybox=True, shadow=True)
    # plt.title()
    # plt.savefig('./norm10-c-ur.png', dpi=600)

In [ ]:
!jupyter nbconvert --to markdown visualize_json_shortterm.ipynb

In [ ]:
!pandoc visualize_json_shortterm.md -o visualize_json_shortterm.pdf --pdf-engine=xelatex  -V geometry:margin=1in 